In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\NTI final project\\Medical-chatbot-GENAI\\Medical-chatbot-GENAI\\research'

In [4]:
import os

os.chdir('../')

In [5]:
%pwd

'c:\\Users\\DELL\\Desktop\\NTI final project\\Medical-chatbot-GENAI\\Medical-chatbot-GENAI'

In [6]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


## Extract data from the PDF file

In [7]:
def load_pdf_fil(data):
    loader = DirectoryLoader(data, glob="*.pdf",
                            loader_cls= PyPDFLoader )
    documents = loader.load()
    
    return documents

In [ ]:
extracted_data = load_pdf_fil('Data/')

In [10]:
# extracted_data

## Splitting and overlapping

In [11]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    docs = text_splitter.split_documents(extracted_data)
    
    return docs

In [12]:
text_chunks = text_split(extracted_data)
print("length of text chunks:", len(text_chunks))

length of text chunks: 5859


## Download the Embedding model from Hugging Face


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

In [17]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
embeddings = download_hugging_face_embedding()

c:\Users\DELL\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back 

In [22]:
query_results = embeddings.embed_query("hello world")
print("lenggth Query results:", len(query_results))

lenggth Query results: 384


In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "testbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    deletion_protection="disabled"  # optional but useful
)


{
    "name": "testbot",
    "metric": "cosine",
    "host": "testbot-78sq2z2.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
import os

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


## Embed each chunk and upsert the embedding into your Pinecone index

In [28]:
from langchain_pinecone import  PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [29]:

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)   

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [31]:
retriever_docs = retriever.invoke("What is the treatment Acne?")

In [32]:
retriever_docs

[Document(id='e8d5b954-93b2-40a2-9d2e-2e5b0a42338e', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='174ce0a1-e104-41b6-bf77-24ad378883b8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='depends upon whether the acne is mild, moderate, or\nsevere.\nDrugs\nTOPICAL DRUGS. Treatment for mild noninflamma-\ntory acne consists of reducing the formation of new\ncomedones with topical tretinoin, benzoyl peroxide, ada-\npalene, or salicylic acid. Tretinoin is especial

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="llama3-70b-8192",
    api_key="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",
    base_url="https://api.groq.com/openai/v1",
    temperature=0.4,        
    max_tokens=500           
)


In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = ("You are an assistant for questions-answering tasks."
                "Use the following pieces of retrieved context to answer the question."
                "the question. If you don't know the answer, just say that you don't know."
                "don't know. Use three sentences maximum and keep the"
                "answer concise and to the point."
                "\n\n"
                "{context}"
                )
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [60]:
question_answer_chain = create_stuff_documents_chain(llm , prompt)
rag_chain = create_retrieval_chain(retriever , question_answer_chain)

In [61]:
response = rag_chain.invoke({"input":"What is Acne?"})
print(response['answer'])

Acne is a common skin disease characterized by pimples on the face, chest, and back, occurring when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [62]:
response = rag_chain.invoke({"input":"What is عبدالظاهر?"})
print(response['answer'])

I don't know.
